In [1]:
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

device = 'cuda'

# Define the MTCNN face detector for cropping
# https://github.com/timesler/facenet-pytorch
# https://www.semanticscholar.org/paper/Joint-Face-Detection-and-Alignment-Using-Multitask-Zhang-Zhang/9e60942aa15670ed9ee03af3c0ae011fa4966b7c

from facenet_pytorch import MTCNN
mtcnn = MTCNN(keep_all=False, select_largest=False, post_process=False, min_face_size=50, device=device)

# Define the emotion recognition network
# https://github.com/av-savchenko/face-emotion-recognition
# https://www.semanticscholar.org/paper/Classifying-Emotions-and-Engagement-in-Online-Based-Savchenko-Savchenko/260d7f95ab8a562f4ff590684ef6a509b8fed316

from hsemotion.facial_emotions import HSEmotionRecognizer
model_name = 'enet_b0_8_best_afew'
fer = HSEmotionRecognizer(model_name=model_name, device=device)
emotion_class_n = 8

C:\Users\Stamina\.hsemotion\enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [2]:
def extract_emotion_scores_from_images(mtcnn_net, images, batch_size=64):

    num_images = len(images)
    all_scores = []

    for start_idx in tqdm(range(0, num_images, batch_size)):
        end_idx = min(start_idx + batch_size, num_images)
        batch_images = images[start_idx:end_idx]

        detections, _ = mtcnn_net.detect(batch_images, landmarks=False)

        detections = np.clip(np.array([detection[0] if detection is not None else [-1, -1, -1, -1] for detection in detections]), 0, 223)

        invalid_images = []

        face_images = []
    
        for i, image in enumerate(batch_images):
            bounding_box = detections[i]
            box = bounding_box.astype(int)
            x1, y1, x2, y2 = box[0:4]
            if x2 == 0:
                face_images.append(np.zeros((224, 224, 3), np.uint8))
                invalid_images.append(i)
                continue
            face_img = image[y1:y2,x1:x2,:]
            face_images.append(face_img)

        _, batch_scores  = fer.predict_multi_emotions(face_images, logits=False)

        for invalid_image_idx in invalid_images:
            batch_scores[invalid_image_idx] = np.zeros_like(batch_scores[0]) + 0.125

        all_scores.extend(batch_scores)
        
    all_scores = np.array(all_scores)
    return all_scores

In [3]:
# Calculate emotion scores for basis images

emotions = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
emotion_scores = dict()
for emotion in emotions:
    emotion_basis_path = f'quant_experiments/basis_aligned/{emotion}_src.jpg'
    emotion_basis_img = np.array(Image.open(emotion_basis_path).resize((224, 224)), dtype=np.uint8) # Resized since emotion network works on 224x224
    emotion_scores[emotion] = extract_emotion_scores_from_images(mtcnn, [emotion_basis_img])[0]

100%|██████████| 1/1 [00:00<00:00, 20.30it/s]


In [39]:
# Calculate emotion score (es) for each configuration

from glob import glob
config_dirs = glob("quant_experiments/transfer/*/*/", recursive = True)

for config_dir in config_dirs:
    inversion_method = config_dir.split('\\')[1]
    emotion = config_dir.split('\\')[2]

    if emotion == 'default':
        continue
    
    image_paths = []
    image_paths.extend(glob(os.path.join(config_dir, '**', '*.jpg'), recursive=True))

    images = []
    for image_path in image_paths:
        images.append(np.array(Image.open(image_path).resize((224, 224)), dtype=np.uint8))

    scores = extract_emotion_scores_from_images(mtcnn, images)
    basis_score = emotion_scores[emotion]

    dot_products = np.dot(scores, basis_score)
    norm_basis = np.linalg.norm(basis_score)
    norms_scores = np.linalg.norm(scores, axis=1)
    cosine_similarities = dot_products / (norm_basis * norms_scores)
    mean_cosine_similarity = np.mean(cosine_similarities)

    print(f'Emotion score (ES) using method {inversion_method} for emotion {emotion} is {mean_cosine_similarity}')

100%|██████████| 31/31 [00:19<00:00,  1.57it/s]


Emotion score (ES) using method hyperstyle for emotion anger is 0.37127119302749634


100%|██████████| 31/31 [00:20<00:00,  1.50it/s]


Emotion score (ES) using method hyperstyle for emotion contempt is 0.7066561579704285


100%|██████████| 31/31 [00:20<00:00,  1.51it/s]


...

Emotion score (ES) using method pSp for emotion neutral is 0.5868839025497437


100%|██████████| 31/31 [00:20<00:00,  1.53it/s]


Emotion score (ES) using method pSp for emotion sadness is 0.6438469886779785


100%|██████████| 31/31 [00:20<00:00,  1.48it/s]

Emotion score (ES) using method pSp for emotion surprise is 0.08573019504547119
